# Computational appendix of [Phys. Rev. Lett. 128, 010403 (2022)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.128.010403)
---
In this notebook we perform a setp-by-step implementation of the hybrid classical-nonsignaling inflation of the bilocal scenario presented in [Phys. Rev. Lett. 128, 010403 (2022)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.128.010403) [[arXiv:2105.09325](https://www.arxiv.org/abs/2105.09325)] and obtain the full network nonlocality witnesses presented there, namely Equations (6-7) and (F1-F2).

Authors: Alejandro Pozas-Kerstjens

Requires: [mosek](https://www.mosek.com/), [numpy](https://numpy.org/), [picos](https://picos-api.gitlab.io/picos/)<=2.2, [qutip](http://qutip.org/), [sympy](https://www.sympy.org/)

Last updated: 21 May, 2021

## Introduction
In this notebook we follow Appendix C of the manuscript in order to implement the linear program arising from application of the [inflation technique](https://www.degruyter.com/document/doi/10.1515/jci-2017-0020/html) to detect whether a tripartite distribution admits a realisation in the bilocal network when one of the sources is assumed to be classical and the other one distributes a general no-signaling resource. We will begin by implementing step by step the linear program in Equation (C6) through direct implementation of Equations (C1-C5), solving the problem, extracting the associated certificate of infeasibility, and deriving from it the full network nonlocality witness in Equation (6) in the manuscript. Next, we will make use of the advanced manipulation of linear programs offered by [picos](https://picos-api.gitlab.io/picos/) in order to simplify the derivation of Equations (7) and (F1-F2).

## Analysing the distribution based on the Elegant Joint Measurement
We begin by importing the necessary libraries and by defining functions that we will use throughout the first part of the notebook, including the definition of the distribution based on the [Elegant Joint Measurement](https://arxiv.org/abs/2006.16694) over noisy singlets, and the function that sets and solves a linear program of the form $$\text{find } \mathbf{x} \text{ such that } A\cdot \mathbf{x} \geq \mathbf{b}.$$

In [1]:
import mosek
import numpy as np
import qutip as qt
import sympy as sp

from itertools import product
from math import pi as π

In [2]:
oA=2; mA=3; oB=4; oC=2; mC=3;

m = np.array([[ 1,  1,  1],
              [ 1, -1, -1],
              [-1,  1, -1],
              [-1, -1,  1]])

l = r'\langle{}'; r = r'\rangle{}';    # For pretty visuals

In [3]:
class ZeroObject(object):
    '''Required for summing lists of constraints'''
    def __add__(self, other):
        return other


def to_indices(position):
    '''
    Returns a linear index for each input-output configuration
    
    :param position: list of inputs and outputs
    :type position: list of ints
    
    :returns: linear index of the configuration
    '''
    a, b1, b2, c1, c2, x, z1, z2 = position
    return (oB**2 * oC**2 * mA * mC**2 * a
            + oB * oC**2 * mA * mC**2 * b1
            + oC**2 * mA * mC**2 * b2
            + oC * mA * mC**2 * c1
            + mA * mC**2 * c2
            + mC**2 * x
            + mC * z1
            + z2)


def EJMtheta(θ, v):
    '''
    Produces the distribution, defined on Eq. (B2) of the manuscript, based on
    the Elegant Joint Measurement
    
    :param θ: Angle of the EJM performed in Bob
    :type θ: float
    :param v: Visibility per singlet
    :type v: float
    
    :returns: array containing the distribution. The entry p[a,b,c,x,z]
              corresponds to p(a,b,c|x,z)
    '''
    ψminus = qt.ket2dm(qt.singlet_state())
    ψminus.dims = [[4], [4]]
    id2 = qt.qeye(2); id4 = qt.qeye(4);

    ρ = qt.tensor([v*ψminus+(1-v)*id4/4, v*ψminus+(1-v)*id4/4])
    ρ.dims = [[2, 4, 2], [2, 4, 2]]

    A = C = [qt.sigmax(), qt.sigmay(), qt.sigmaz()]
    ηs = [1/np.sqrt(3), -1/np.sqrt(3), -1/np.sqrt(3), 1/np.sqrt(3)]
    ϕs = [π/4, -π/4, 3*π/4, -3*π/4]

    plusm  = np.zeros((4,2), dtype=complex)
    minusm = np.zeros((4,2), dtype=complex)
    ejm    = [0, 0, 0, 0]
    B      = []

    for j in range(oB):
        plusm[j]  = [np.sqrt((1+ηs[j])/2)*np.exp(-1j*ϕs[j]/2),
                     np.sqrt((1-ηs[j])/2)*np.exp(1j*ϕs[j]/2)]
        minusm[j] = [np.sqrt((1-ηs[j])/2)*np.exp(-1j*ϕs[j]/2),
                     -np.sqrt((1+ηs[j])/2)*np.exp(1j*ϕs[j]/2)]
        ejm[j]    = qt.Qobj((
        ((np.sqrt(3)+np.exp(1j*θ))/(2*np.sqrt(2)))*np.kron(plusm[j],minusm[j])
        + ((np.sqrt(3)-np.exp(1j*θ))/(2*np.sqrt(2)))*np.kron(minusm[j],plusm[j])
        ))
        B.append(qt.ket2dm(ejm[j]))

    p = np.zeros((oA,oB,oC,mA,mC))
    for a in range(oA):
        for b in range(oB):
            for c in range(oC):
                for x in range(mA):
                    for z in range(mC):
                        p[a,b,c,x,z] = np.real(ρ.overlap(
                       qt.tensor([id2+(-1)**a*A[x], B[b], id2+(-1)**c*C[z]]))/4)
    return p


def set_lp(A, b, solve=False):
    '''
    Sets and (optionally) solves the linear program A.x >= b
    
    :param A: Matrix of the coefficients of each variable in every constraint
    :type A:  2-D array
    :param b: Vector of lower bounds for each constraint
    :type b:  1-D array
    :param solve: (Optionally) solve the linear program
    :type solve:  Bool
    
    :returns: If solve=False, the MOSEK problem.
              If solve=True and the LP is feasible, an array with the optimal
              solution and None.
              If solve=True and the LP is infeasible, an array with None and a
              certificate of infeasibility
    '''
    # Make mosek environment
    task = mosek.Env().Task(0, 0)

    # Generate the placeholder for the constraints.
    # The constraints will initially have no bounds.
    task.appendcons(n_constraints)

    # Generate the variables in the problem.
    # The variables will initially be fixed at zero (x=0).
    task.appendvars(dims)

    for j in range(dims):
        # Set the bounds on variable j. We allow them to be free variables
        task.putvarbound(j, mosek.boundkey.fr, 0, 0)

    for i in range(n_constraints):
        # Set the constraints.
        # Select the variables that are involved and the associated coefficients
        indices = np.nonzero(LPA[i,:])[0]
        
        task.putarow(i,                  # Variable (column) index.
                     indices,            # Row index of non-zeros in column j.
                     LPA[i,indices])     # Non-zero Values of column j.
        
        # Set lower bounds
        task.putconbound(i, mosek.boundkey.lo, LPb[i], 0)

    if solve:
        # Solve the problem
        task.optimize()

        # Get status information about the solution
        solsta = task.getsolsta(mosek.soltype.bas)

        if (solsta == mosek.solsta.optimal):
            xx = [0.] * dims
            task.getxx(mosek.soltype.bas, xx)    # Request the basic solution
            print('Optimal solution: ')
            for i in range(dims):
                print('x[' + str(i) + ']=' + str(xx[i]))
            return xx, None
        elif (solsta == mosek.solsta.dual_infeas_cer or
              solsta == mosek.solsta.prim_infeas_cer):
            y = [0.] * n_constraints
            task.gety(mosek.soltype.bas, y)    # Request the certificate
            print('Primal or dual infeasibility certificate found')
            return None, y
        elif solsta == mosek.solsta.unknown:
            print('Unknown solution status')
        else:
            print('Other solution status')
    else:
        return task

Inflation is a general procedure to derive constraints that a probability distribution must satisfy in order to be possible to produce it in a particular causal structure. These constraints are produced by analysing the distributions that could be generated by having access to multiple copies of the elements of the causal structure. For instance, if one wanted to analyse the kind of distributions that could be generated in the bilocal network below

<img src='bilocality.png' width='250' height='250'>

where $\lambda$ represents a source of classical randomness and $NS$ is a source of a general no-signaling resource. If given access to multiple copies of the measurement devices $A$, $B$ and $C$, and of the sources $\lambda$ and $NS$, one may wonder what properties the distribution in the following inflation

<img src='bilocalitynsc.png' width='250' height='250'>

would satisfy. Note that, since $\lambda$ emits classical randomness that can be cloned, we have simply cloned the information sent to Bob in order to send the same information to both copies now available. On the contrary, the information encoded in non-classical systems (both quantum and supra-quantum) cannot be cloned, and thus we cannot copy the information sent by $NS$, but we can consider copies of the source itself.

### Deriving Equation 6
Before analysing the characteristics of the distributions that can be generated in the inflation, let us initialise a few useful quantities: the number of variables in the problem, the number of constraints of each type that we will encounter, the matrix $A$ and the vector $\mathbf{b}$, and a symbolic representation of the original probability distribution. The matrix $A$ has as many rows as constraints, and as many columns as variables in the problem, while the length of the vector $\mathbf{b}$ equates to the number of constraints that will be imposed on the variables.

In [4]:
dims = oA*oB*oB*oC*oC*mA*mC*mC

n_pos_const = dims
n_norm_const = mA*mC*mC
n_nsA_const = (mA-1)*oB*oB*oC*oC*mC*mC
n_nsC_const = 2*(mC-1)*oA*oB*oB*oC*mA*mC
n_inf_const = (dims - oA*oB*oC*mA*mC)//2  # All except when b1=b2, c1=c2, z1=z2
n_marginals_const = oA*oB*oC*oC*mA*mC*mC
n_constraints = n_pos_const + 2*(n_norm_const + n_nsA_const + n_nsC_const
                                 + n_inf_const + n_marginals_const)

LPA = np.zeros((n_constraints, dims))
LPb = np.zeros((n_constraints))
symbolicb = np.zeros((n_constraints), dtype=object)

symp = np.zeros((oA,oB,oC,mA,mC), dtype=object)
for a in range(oA):
    for b in range(oB):
        for c in range(oC):
            for x in range(mA):
                for z in range(mC):
                    symp[a,b,c,x,z] = sp.symbols('p('+str(a)+str(b)+str(c)+
                                                 '|'+str(x)+str(z)+')')

The first property that the probability distribution (and any probability distribution) must satisfy is that its elements are all positive. This is Equation C1 in the manuscript. If we want these to be the first constraints imposed on the vector $\mathbf{x}$, this implies that the top-left block of the matrix $A$ is $\mathbb{1}_{\text{len}(\mathbf{x}),\text{len}(\mathbf{x})}$, and the first $\text{len}(\mathbf{x})$ elements of $\mathbf{b}$ are zero.

In [5]:
# Positivity
np.fill_diagonal(LPA[:dims, :dims], 1)
i = n_pos_const

Second, for every possible choice of $x$, $z^1$ and $z^2$, the distribution is normalised (see Equation C2 in the manuscript). These are equality constraints, which we impose using two rows in $A$ for each: one encoding $\sum_{a,b^1,b^2,c^1,c^2}p_\text{inf}(a,b^1,b^2,c^1,c^2|x,z^1,z^2) \geq 1$ and another encoding $-\sum_{a,b^1,b^2,c^1,c^2}p_\text{inf}(a,b^1,b^2,c^1,c^2|x,z^1,z^2) \geq -1$.

In [6]:
# Normalisation
for x, z1, z2 in product(range(mA), range(mC), range(mC)):
    outputs = np.array(list(product(range(oA), range(oB), range(oB),
                                    range(oC), range(oC))))
    inputs = np.array([[x, z1, z2],]*len(outputs))
    all_indices = np.hstack([outputs, inputs])
    positions = np.apply_along_axis(to_indices, 1, all_indices)
    LPA[i, positions] = 1
    LPb[i] = 1
    symbolicb[i] = 1
    LPA[i+n_norm_const, positions] = -1
    LPb[i+n_norm_const] = -1
    symbolicb[i+n_norm_const] = -1
    i += 1
if i == n_pos_const + n_norm_const:
    i = n_pos_const + 2*n_norm_const
else:
    print('There is a counting problem. You may have forgotten constraints')

Next, in Equation C3 one finds the no-signaling constraints, which ensure that the parties cannot communicate their choice of input to the rest. This is achieved by imposing that the marginal over a party's outcome is insensitive to the input chosen by that party, this is, that $\sum_{a}p_\text{inf}(a,b^1,b^2,c^1,c^2|x,z^1,z^2) = \sum_{a}p_\text{inf}(a,b^1,b^2,c^1,c^2|x',z^1,z^2)$ for any choice of $b^1$, $b^2$, $c^1$, $c^2$, $x$, $x'$, $z^1$ and $z^2$, and analogously for parties Charlie${}^1$ and Charlie${}^2$. If Bob had a choice of input as well, similar constraints should be imposed.

In [7]:
# No-signaling A
for x,b1,b2,c1,c2,z1,z2 in product(range(1, mA), range(oB), range(oB),
                                   range(oC), range(oC), range(mC), range(mC)):
    index1 = to_indices([0,b1,b2,c1,c2,0,z1,z2])
    index2 = to_indices([1,b1,b2,c1,c2,0,z1,z2])
    index3 = to_indices([0,b1,b2,c1,c2,x,z1,z2])
    index4 = to_indices([1,b1,b2,c1,c2,x,z1,z2])
    LPA[i, [index1, index2]] = 1
    LPA[i, [index3, index4]] = -1
    LPA[i+n_nsA_const, [index1, index2]] = -1
    LPA[i+n_nsA_const, [index3, index4]] = 1
    i += 1
if i == n_pos_const + 2*n_norm_const+n_nsA_const:
    i = n_pos_const + 2*(n_norm_const+n_nsA_const)
else:
    print('There is a counting problem. You may have forgotten constraints')
    
# No-signaling C1
for z,a,b1,b2,c2,x,z2 in product(range(1, mC), range(oA), range(oB), range(oB),
                                 range(oC), range(mA), range(mC)):
    index1 = to_indices([a,b1,b2,0,c2,x,0,z2])
    index2 = to_indices([a,b1,b2,1,c2,x,0,z2])
    index3 = to_indices([a,b1,b2,0,c2,x,z,z2])
    index4 = to_indices([a,b1,b2,1,c2,x,z,z2])
    LPA[i, [index1, index2]] = 1
    LPA[i, [index3, index4]] = -1
    LPA[i+int(n_nsC_const/2), [index1, index2]] = -1
    LPA[i+int(n_nsC_const/2), [index3, index4]] = 1
    i += 1
if i == n_pos_const + 2*(n_norm_const+n_nsA_const)+n_nsC_const/2:
    i = int(n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const/2))
else:
    print('There is a counting problem. You may have forgotten constraints')

# No-signaling C2
for z,a,b1,b2,c1,x,z1 in product(range(1, mC), range(oA), range(oB), range(oB),
                                 range(oC), range(mA), range(mC)):
    index1 = to_indices([a,b1,b2,c1,0,x,z1,0])
    index2 = to_indices([a,b1,b2,c1,1,x,z1,0])
    index3 = to_indices([a,b1,b2,c1,0,x,z1,z])
    index4 = to_indices([a,b1,b2,c1,1,x,z1,z])
    LPA[i, [index1, index2]] = 1
    LPA[i, [index3, index4]] = -1
    LPA[i+int(n_nsC_const/2), [index1, index2]] = -1
    LPA[i+int(n_nsC_const/2), [index3, index4]] = 1
    i += 1
if i == n_pos_const + 2*(n_norm_const+n_nsA_const)+3*n_nsC_const/2:
    i = int(n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const))
else:
    print('There is a counting problem. You may have forgotten constraints')

The crucial part to inflation is the symmetry of the probability distribution that is inherited by the fact that the elements in the inflated network are exact copies of the elements in the original network. Since $B^1$ is identical to $B^2$, $C^1$ is identical to $C^2$, $NS^1$ is identical to $NS^2$, and $\lambda$ sends the exact same information to both $B^1$ and $B^2$, the distributions that are generated in the inflated network must be insensitive to the swap $1\leftrightarrow 2$. This is captured by Equation C4 in the manuscript, namely $p_\text{inf}(a,b^1,b^2,c^1,c^2|x,z^1,z^2) = p_\text{inf}(a,b^2,b^1,c^2,c^1|x,z^2,z^1)$.

In [8]:
# Inflation
used_indices = []
for a,b1,b2,c1,c2,x,z1,z2 in product(range(oA), range(oB), range(oB), range(oC),
                                     range(oC), range(mA), range(mC), range(mC)):
    index1 = to_indices([a,b1,b2,c1,c2,x,z1,z2])    # Original distribution
    used_indices.append(index1)
    index2 = to_indices([a,b2,b1,c2,c1,x,z2,z1])    # Swapping B1C1Z1<->B2C2Z2
    if (index1 != index2) and index2 not in used_indices:
        LPA[i, index1] = 1
        LPA[i, index2] = -1
        LPA[i+n_inf_const, index1] = -1
        LPA[i+n_inf_const, index2] = 1
        i += 1
if i == n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const)+n_inf_const:
    i = n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const+n_inf_const)
else:
    print('There is a counting problem. You may have forgotten constraints')

Finally, if the outcome of one of the copies of Bob is disregarded, one notices that Alice, the remaining Bob, and its associated Charlie, are precisely reproducing the original network and thus their measurement outcomes will follow the statistics dictated by the original distribution. Moreover, the remaining Charlie is also measuring on his share of the original resource. This means that $\sum_{b^2}p_\text{inf}(a,b^1,b^2,c^1,c^2|x,z^1,z^2) = p_\text{orig}(a,b^1,c^1|x,z^1)p_\text{orig}(c^2|z^2)$, captured in Equation C5 in the manuscript. Note that one could also trace Bob${}^1$ instead, having new constraints. These can, however, be derived from the ones implemented below and the inflation constraints above.

In [9]:
# Matching original probabilities
orig = EJMtheta(π/4, 0.9)
for a,b,c1,c2,x,z1,z2 in product(range(oA), range(oB), range(oC), range(oC),
                                 range(mA), range(mC), range(mC)):
    indices = [to_indices([a,b,b2,c1,c2,x,z1,z2]) for b2 in range(oB)]
    LPA[i, indices] = 1
    LPb[i] = orig[a,b,c1,x,z1]*sum(sum(orig[:,:,c2,0,z2]))
    symbolicb[i] = symp[a,b,c1,x,z1]*sum(sum(symp[:,:,c2,0,z2]))
    LPA[i+n_marginals_const, indices] = -1
    LPb[i+n_marginals_const] = -orig[a,b,c1,x,z1]*sum(sum(orig[:,:,c2,0,z2]))
    symbolicb[i+n_marginals_const] = -symp[a,b,c1,x,z1]*sum(sum(symp[:,:,c2,0,z2]))
    i += 1
if i == n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const+n_inf_const)+n_marginals_const:
    i = n_pos_const + 2*(n_norm_const+n_nsA_const+n_nsC_const+n_inf_const+n_marginals_const)
else:
    print('There is a counting problem. You may have forgotten constraints')

Now that all the constraints are captured in $A$ and $\mathbf{b}$, we solve the linear program in Equation C6 in the manuscript.

In [10]:
[xsol, y] = set_lp(LPA, LPb, True)

Primal or dual infeasibility certificate found


The problem is infeasible, i.e., it has no solution. Since all the derived constraints are conditions satisfied if a distribution $p$ can be generated in the bilocal scenario, this result means that the distribution we have tested, $p^{0.9}_{\pi/4}$, cannot be generated in a bilocal scenario where the $A-B$ source is classical and the $B-C$ source is no-signaling.

[Farkas' lemma](https://en.wikipedia.org/wiki/Farkas'_lemma) guarantees that, if no solution exists to a linear program of the form $A\cdot\mathbf{x}\geq \mathbf{b}$, then there exists a vector $\mathbf{y}$ such that $\mathbf{y}\cdot A=\mathbf{0}$ and $\mathbf{y}\cdot\mathbf{b} > 0$, which is called a certificate. We can extract such certificate from the problem solved, and check that it indeed satisfies its properties.

In [11]:
y = np.array(y)
y[abs(y) < 1e-6] = 0
print('Σ|y.A| = ', round(sum(abs(y @ LPA)), 4))
print('y.b = ', round(y @ LPb, 4))

Σ|y.A| =  0.0
y.b =  0.0012


Notably, since the dependence on the distribution under scrutiny is exclusively contained in the vector $\mathbf{b}$, the extracted certificate $\mathbf{y}$ will certify the infeasibility not just of the tested $p_\text{obs}$, but also of any distribution for which $\mathbf{y}\cdot\mathbf{b}(p_\text{obs}) > 0$. We can thus create an abstract $\mathbf{b}(p_\text{obs})$ (in fact, we have been filling it at the same time we were filling $\mathbf{b}$), and evaluate $\mathbf{y}\cdot\mathbf{b}(p_\text{obs})$.

In [12]:
expA = np.array(sp.symbols([l + 'A_{' + str(x+1) + '}' + r for x in range(mA)]))
expB = np.array(sp.symbols([l + 'B_{' + str(y+1) + '}' + r for y in range(3)]))
expC = np.array(sp.symbols([l + 'C_{' + str(z+1) + '}' + r for z in range(mC)]))
expAB = np.array(sp.symbols([[l + 'A_{' + str(x+1) + '}B_{' + str(y+1) + '}' + r
                              for y in range(3)] for x in range(mA)]))
expAC = np.array(sp.symbols([[l + 'A_{' + str(x+1) + '}C_{' + str(z+1) + '}' + r
                              for z in range(mC)] for x in range(mA)]))
expBC = np.array(sp.symbols([[l + 'B_{' + str(y+1) + '}C_{' + str(z+1) + '}' + r
                              for z in range(mC)] for y in range(3)]))
expABC = np.array(sp.symbols([[[l+'A_{'+str(x)+'}B_{'+str(y)+'}C_{'+str(z)+'}'+r
                                for z in range(1, mC+1)]
                               for y in range(1, 4)]
                              for x in range(1, mA+1)]))

In [13]:
probs = [symp[a,b,c,x,z]
         for a, b, c, x, z in product(range(oA), range(oB), range(oC),
                                      range(mA), range(mC))]
corrs = [(1 + (-1)**a*expA[x] + m[b]@expB + (-1)**c*expC[z]
          + (-1)**a*m[b]@expAB[x,:] + (-1)**c*m[b]@expBC[:,z]
          + (-1)**(a+c)*expA[x]*expC[z]
          + (-1)**(a+c)*m[b]@expABC[x,:,z])/16
         for a, b, c, x, z in product(range(oA), range(oB), range(oC),
                                      range(mA), range(mC))]

In [14]:
sp.nsimplify(
    sp.expand(
            (8*y.T @ symbolicb).subs(zip(probs, corrs))), tolerance=1e-6)

-\langle{}A_{1}B_{2}C_{3}\rangle{} + \langle{}A_{1}B_{2}\rangle{}*\langle{}C_{3}\rangle{} + \langle{}A_{2}B_{2}C_{3}\rangle{}*\langle{}C_{3}\rangle{} - \langle{}A_{2}B_{2}\rangle{} + \langle{}C_{3}\rangle{}**2 - 1

This expression, which is positive for full-network-nonlocal distributions, is precisely Equation 6 in the manuscript. There, it is formulated in such a way that full network nonlocality is witnessed when the inequality is violated.

## Simplifying the implementation
Writing explicitly $A$ and $\mathbf{b}$ for an inflation problem may be tedious in some situations, especially when considering large networks or many copies of the network elements. However, it is not necessary to generate them explicitly if using libraries such as [picos](https://picos-api.gitlab.io/picos/). In the following, we resort to picos for generating and solving the linear programs necessitated for deriving Equations (6-7) and (F1-F2).

In [15]:
import picos as pic

### Deriving (again) Equation 6

In [16]:
def set_problem_C_NS(prob_dist, solve=False):
    '''
    Sets and (optionally) solves the LP associated to the feasibility of a
    tripartite distribution admitting an inflation of the form of Figure 3 in
    the manuscript.
    
    :param prob_dist: The probability distribution under scrutiny
    :type prob_dist:  5-D array
    :param solve: (Optionally) solve the linear program
    :type solve:  Bool
    
    :returns: If solve=False, the PICOS problem.
              If solve=True and the LP is feasible, an array with the optimal
              solution and [None, None].
              If solve=True and the LP is infeasible, an array with None and a
              certificate of infeasibility, split into the components associated
              to normalisation constraints, and the components associated to
              constraints of matching with observed probabilities.    
    '''
    oA, oB, oC, mA, mC = prob_dist.shape
    probabilities = pic.RealVariable('pinf',
                                   shape=oA*oB*oB*oC*oC*mA*mC*mC,
                                   lower=0)
    pinf = np.reshape(probabilities, (oA,oB,oB,oC,oC,mA,mC,mC))

    # Normalisation
    norm_const = [probsum == 1
                  for probsum in np.sum(pinf, (0, 1, 2, 3, 4)
                                       ).flatten().tolist()]

    # No-signaling A
    nsA_const = [[] for _ in range(mA-1)]
    nsA_0_terms = np.sum(pinf[:,:,:,:,:,0,:,:], 0)
    for x in range(1, mA):
        nsA_x_terms = np.sum(pinf[:,:,:,:,:,x,:,:], 0)
        nsA_const[x-1] = [term == 0
                          for term in (nsA_0_terms
                                       - nsA_x_terms).flatten().tolist()]
    nsA_const = sum(nsA_const, ZeroObject())

    # No-signaling C
    nsC1_const = [[] for _ in range(mC-1)]
    nsC1_0_terms = np.sum(pinf[:,:,:,:,:,:,0,:], 3)
    nsC2_const = [[] for _ in range(mC-1)]
    nsC2_0_terms = np.sum(pinf[:,:,:,:,:,:,:,0], 4)
    for z in range(1, mC):
        nsC1_z_terms = np.sum(pinf[:,:,:,:,:,:,z,:], 3)
        nsC1_const[z-1] = [term == 0
                           for term in (nsC1_0_terms
                                        - nsC1_z_terms).flatten().tolist()]
        nsC2_z_terms = np.sum(pinf[:,:,:,:,:,:,:,z], (4))
        nsC2_const[z-1] = [term == 0
                           for term in (nsC2_0_terms
                                        - nsC2_z_terms).flatten().tolist()]

    nsC_const = sum(nsC1_const, ZeroObject()) + sum(nsC2_const, ZeroObject())

    # Inflation
    inf_const = [term == 0
                 for term in (pinf-np.transpose(pinf, [0,2,1,4,3,5,7,6])
                             ).flatten().tolist()]

    # Matching original probabilities
    marginal = np.zeros((oA,oB,oC,oC,mA,mC,mC))
    for c2 in range(oC):
        for z2 in range(mC):
            marginal[:,:,:,c2,:,:,z2] = prob_dist*np.sum(prob_dist[:,:,c2,0,z2])

    marginal_const = [term == 0
                      for term in (np.sum(pinf, 2)-marginal).flatten().tolist()]
    
    # Set up the problem
    problem = pic.Problem()
    for const in norm_const+nsC_const+nsA_const+inf_const+marginal_const:
        problem.add_constraint(const)

    # Solve if requested
    if solve:
        try:
            problem.solve(solver='mosek')
            print('Optimal solution found')
            return [np.reshape(probabilities.value,
                               (oA,oB,oB,oC,oC,mA,mC,mC)), [None, None]]
        except pic.SolutionFailure:
            print('The problem is infeasible. Getting certificate...')
            problem.solve(solver='mosek', primals=False)            
            print('Primal or dual infeasibility certificate found')
            ynorm     = np.array([const.dual for const in norm_const])
            ymarginal = np.array([const.dual for const in marginal_const])
            ynorm[abs(ynorm) < 1e-6] = 0
            ymarginal[abs(ymarginal) < 1e-6] = 0
            return [None, [ynorm, ymarginal]]
    else:
        return problem

Note that, in case of finding an infeasible problem, the only sectors of $\mathbf{b}$ that are returned are those corresponding to the normalisation and matching constraints, since for the rest the corresponding entries in $\mathbf{b}$ are all zero.

In [17]:
[pinf, [ynorm, ymarginal]] = set_problem_C_NS(EJMtheta(π/4, 0.9), solve=True)

The problem is infeasible. Getting certificate...
Primal or dual infeasibility certificate found


Now that the infeasibility has been detected, we generate the inequality

In [18]:
marginalsymb = np.zeros((oA,oB,oC,oC,mA,mC,mC), dtype=object)
for c2 in range(oC):
    for z2 in range(mC):
        marginalsymb[:,:,:,c2,:,:,z2] = symp * np.sum(symp[:,:,c2,0,z2])
            
sp.nsimplify(sp.expand((
                        8*(ymarginal @ marginalsymb.flatten() + sum(ynorm))
                        ).subs(zip(probs, corrs))), tolerance=1e-6)

-\langle{}A_{2}B_{3}C_{1}\rangle{} + \langle{}A_{2}B_{3}\rangle{}*\langle{}C_{1}\rangle{} + \langle{}A_{3}B_{3}C_{1}\rangle{}*\langle{}C_{1}\rangle{} - \langle{}A_{3}B_{3}\rangle{} + \langle{}C_{1}\rangle{}**2 - 1

This is the same witness as above, if performing the relabeling $P_i\rightarrow P_{i - 1}$ for every party $P$, with $P_0=P_3$.

## Deriving Equation 7
Equation 7 in the manuscript is obtained when contrasting $p^v_\theta$ against a hybrid bilocal model where the classical source is shared by Bob and Charlie. We can easily set up the corresponding inflation and solve the associated linear program.

In [19]:
def set_problem_NS_C(prob_dist, solve=False):
    '''
    Sets and (optionally) solves the LP associated to the feasibility of a
    tripartite distribution admitting an inflation of the form of Figure 3 in
    the manuscript, with the roles of Alice and Charlie swapped.
    
    :param prob_dist: The probability distribution under scrutiny
    :type prob_dist:  5-D array
    :param solve: (Optionally) solve the linear program
    :type solve:  Bool
    
    :returns: If solve=False, the PICOS problem.
              If solve=True and the LP is feasible, an array with the optimal
              solution and [None, None].
              If solve=True and the LP is infeasible, an array with None and a
              certificate of infeasibility, split into the components associated
              to normalisation constraints, and the components associated to
              constraints of matching with observed probabilities.    
    '''
    oA, oB, oC, mA, mC = prob_dist.shape
    probabilities = pic.RealVariable('pinf',
                                   shape=oA*oA*oB*oB*oC*mA*mA*mC,
                                   lower=0)
    pinf = np.reshape(probabilities, (oA,oA,oB,oB,oC,mA,mA,mC))

    norm_const = [probsum == 1
                  for probsum in np.sum(pinf,(0, 1, 2, 3, 4)
                                       ).flatten().tolist()]

    nsA1_const = [[] for _ in range(mA-1)]
    nsA1_0_terms = np.sum(pinf[:,:,:,:,:,0,:,:], 0)
    nsA2_const = [[] for _ in range(mA-1)]
    nsA2_0_terms = np.sum(pinf[:,:,:,:,:,:,0,:], 1)
    for x in range(1, mA):
        nsA1_x_terms = np.sum(pinf[:,:,:,:,:,x,:,:], 0)
        nsA1_const[x-1] = [term == 0
                           for term in (nsA1_0_terms
                                        - nsA1_x_terms).flatten().tolist()]
        nsA2_x_terms = np.sum(pinf[:,:,:,:,:,:,x,:], 1)
        nsA2_const[x-1] = [term == 0
                           for term in (nsA2_0_terms
                                        - nsA2_x_terms).flatten().tolist()]

    nsA_const = sum(nsA1_const, ZeroObject()) + sum(nsA2_const, ZeroObject())
    
    nsC_const = [[] for _ in range(mC-1)]
    nsC_0_terms = np.sum(pinf[:,:,:,:,:,:,:,0], 4)
    for z in range(1, mC):
        nsC_z_terms = np.sum(pinf[:,:,:,:,:,:,:,z], 4)
        nsC_const[z-1] = [term == 0
                          for term in (nsC_0_terms
                                       - nsC_z_terms).flatten().tolist()]
    nsC_const = sum(nsC_const, ZeroObject())

    inf_const = [term == 0
                 for term in (pinf-np.transpose(pinf, [1,0,3,2,4,6,5,7])
                             ).flatten().tolist()]

    marginal = np.zeros((oA,oA,oB,oC,mA,mA,mC))
    for a2 in range(oA):
        for x2 in range(mA):
            marginal[:,a2,:,:,:,x2,:] = prob_dist*np.sum(prob_dist[a2,:,:,x2,0])

    marginal_const = [term == 0
                      for term in (np.sum(pinf, 2)-marginal).flatten().tolist()]
    
    problem = pic.Problem()
    for const in norm_const+nsC_const+nsA_const+inf_const+marginal_const:
        problem.add_constraint(const)

    if solve:
        try:
            problem.solve(solver='mosek')
            print('Optimal solution found')
            return [np.reshape(probabilities.value,
                               (oA,oA,oB,oB,oC,mA,mA,mC)), None]
        except pic.SolutionFailure:
            print('The problem is infeasible. Getting certificate...')
            problem.solve(solver='mosek', primals=False)            
            print('Primal or dual infeasibility certificate found')
            ynorm     = np.array([const.dual for const in norm_const])
            ymarginal = np.array([const.dual for const in marginal_const])
            ynorm[abs(ynorm) < 1e-6] = 0
            ymarginal[abs(ymarginal) < 1e-6] = 0
            return [None, [ynorm, ymarginal]]
    else:
        return problem

In [20]:
[pinf, [ynorm, ymarginal]] = set_problem_NS_C(EJMtheta(π/4, 0.9), solve=True)

The problem is infeasible. Getting certificate...
Primal or dual infeasibility certificate found


In [21]:
marginalsymb = np.zeros((oA,oA,oB,oC,mA,mA,mC), dtype=object)
for a2 in range(oA):
    for x2 in range(mA):
        marginalsymb[:,a2,:,:,:,x2,:] = symp * np.sum(symp[a2,:,:,x2,0])

sp.nsimplify(sp.expand((
                        8*(ymarginal @ marginalsymb.flatten() + sum(ynorm))
                        ).subs(zip(probs, corrs))), tolerance=1e-6)

-\langle{}A_{3}B_{1}C_{1}\rangle{}*\langle{}A_{3}\rangle{} - \langle{}A_{3}B_{1}C_{2}\rangle{} + \langle{}A_{3}\rangle{}**2 + \langle{}A_{3}\rangle{}*\langle{}B_{1}C_{2}\rangle{} + \langle{}B_{1}C_{1}\rangle{} - 1

This is the same witness as that in Equation 8 in the manuscript, if performing the relabeling $P_i\rightarrow P_{i + 1}$ for every party $P$, with $P_4=P_1$. Note that the distribution $p_{\pi/4}^{0.9}$ violates both witnesses of incompatibility with $\lambda-NS$ models and with $NS-\lambda$ models, and thus, _it is fully network nonlocal_.

# Witnesses for the distribution based on the Bell state measurement
Finally, we provide full network Bell witnesses for the bilocal scenario which admits a quantum violation based on a protocol that is comparatively convenient for implementation in optical systems. Instead of Bob performing an Elegant Joint Measurement, he performs a three-outcome, imperfect Bell state measurement where he can perfectly distinguish the states $\phi^{\pm}=\frac{|00\rangle\pm|11\rangle}{\sqrt{2}}$, but cannot distinguish between $\psi^{\pm}=\frac{|01\rangle\pm|10\rangle}{\sqrt{2}}$. The procedure is exactly the same as that followed in the remainder of the notebook.

In [22]:
oA=2; oB=3; oC=2; mA=2; mC=2;

def BSMprob(v):
    '''
    Produces the distribution, defined on Appendix F of the manuscript, based on
    the imperfect Bell state measurement
    
    :param v: Visibility per singlet
    :type v: float
    
    :returns: array containing the distribution. The entry p[a,b,c,x,z]
              corresponds to p(a,b,c|x,z)
    '''
    ϕplus  = qt.ket2dm(qt.bell_state('00'))
    ϕminus = qt.ket2dm(qt.bell_state('01'))
    id2 = qt.qeye(2); id4 = qt.qeye(4)
    id4.dims = [[2, 2], [2, 2]]

    ρ = qt.tensor([v*ϕplus+(1-v)*id4/4, v*ϕplus+(1-v)*id4/4])
    ρ.dims = [[2, 4, 2], [2, 4, 2]]

    A = [qt.sigmax(), qt.sigmaz()]
    B = [ϕplus, ϕminus, id4 - ϕplus - ϕminus]
    for proj in B:
        proj.dims = [[4], [4]]
    C = [(qt.sigmaz() + qt.sigmax()) / np.sqrt(2),
         (qt.sigmaz() - qt.sigmax()) / np.sqrt(2)]

    probBSM = np.zeros((oA,oB,oC,mA,mC))
    for a in range(oA):
        for b in range(oB):
            for c in range(oC):
                for x in range(mA):
                    for z in range(mC):
                        probBSM[a,b,c,x,z] = np.real(ρ.overlap(
                           qt.tensor([id2+(-1)**a*A[x],
                                      B[b],
                                      id2+(-1)**c*C[z]])
                                     )/4)
    return probBSM

In [23]:
symp = np.zeros((oA,oB,oC,mA,mC), dtype='object')
for a in range(oA):
    for b in range(oB):
        for c in range(oC):
            for x in range(mA):
                for z in range(mC):
                    symp[a,b,c,x,z] = sp.symbols('p('+str(a)+str(b)+str(c)+
                                                 '|'+str(x)+str(z)+')')

expA = np.array(sp.symbols([l + 'A_{' + str(x) + '}' + r for x in range(mA)]))
expB = np.array(sp.symbols([l + 'B_{' + str(y) + '}' + r for y in range(2)]))
expC = np.array(sp.symbols([l + 'C_{' + str(z) + '}' + r for z in range(mC)]))
expAB = np.array(sp.symbols([[l + 'A_{' + str(x) + '}B_{' + str(y) + '}' + r
                              for y in range(2)]
                             for x in range(mA)]))
expAC = np.array(sp.symbols([[l + 'A_{' + str(x) + '}C{' + str(z) + '}' + r
                              for z in range(mC)]
                             for x in range(mA)]))
expBC = np.array(sp.symbols([[l + 'B_{' + str(y) + '}C_{' + str(z) + '}' + r
                              for z in range(mC)]
                             for y in range(2)]))
expABC = np.array(sp.symbols([[[l+'A_{'+str(x)+'}B_{'+str(y)+'}C_{'+str(z)+'}'+r
                                for z in range(mC)]
                               for y in range(2)]
                              for x in range(mA)]))

In [24]:
probs = [symp[a,b,c,x,z] for a, b, c, x, z in product(range(oA), range(oB),
                                                      range(oC), range(mA),
                                                      range(mC))]
corrs = np.zeros((oA, oB, oC, mA, mC), dtype=object)
corrs[:,0,:,:,:] = [[[[(1 + (-1)**a*expA[x]+expB[0]+2*expB[1]+(-1)**c*expC[z]
                     + (-1)**a*(expAB[x,0] + 2*expAB[x,1])
                     + (-1)**c*(expBC[0,z] + 2*expBC[1,z])
                     + (-1)**(a+c)*expA[x]*expC[z]
                     + (-1)**(a+c)*(expABC[x,0,z] + 2*expABC[x,1,z]))/16
                    for z in range(mC)] for x in range(mA)]
                     for c in range(oC)] for a in range(oA)]
corrs[:,1,:,:,:] = [[[[(1 + (-1)**a*expA[x]+expB[0]-2*expB[1]+(-1)**c*expC[z]
                     + (-1)**a*(expAB[x,0] - 2*expAB[x,1])
                     + (-1)**c*(expBC[0,z] - 2*expBC[1,z])
                     + (-1)**(a+c)*expA[x]*expC[z]
                     + (-1)**(a+c)*(expABC[x,0,z] - 2*expABC[x,1,z]))/16
                    for z in range(mC)] for x in range(mA)]
                     for c in range(oC)] for a in range(oA)]
corrs[:,2,:,:,:] = [[[[(1 + (-1)**a*expA[x] - expB[0] + (-1)**c*expC[z]
                     - (-1)**a*expAB[x,0] - (-1)**c*expBC[0,z]
                     + (-1)**(a+c)*expA[x]*expC[z]
                     - (-1)**(a+c)*expABC[x,0,z])/8
                    for z in range(mC)] for x in range(mA)]
                     for c in range(oC)] for a in range(oA)]

corrs = [corrs[a,b,c,x,z] for a, b, c, x, z in product(range(oA), range(oB),
                                                       range(oC), range(mA),
                                                       range(mC))]

### Equation F1

In [25]:
[pinf, [ynorm, ymarginal]] = set_problem_C_NS(BSMprob(0.9212), solve=True)

marginalsymb = np.zeros((oA,oB,oC,oC,mA,mC,mC), dtype=object)
for c2 in range(oC):
    for z2 in range(mC):
        marginalsymb[:,:,:,c2,:,:,z2] = symp * np.sum(symp[:,:,c2,0,z2])
           
sp.nsimplify(sp.expand((
                        8*(ymarginal @ marginalsymb.flatten() + sum(ynorm))
                        ).subs(zip(probs, corrs))), tolerance=1e-6)

The problem is infeasible. Getting certificate...
Primal or dual infeasibility certificate found


2*\langle{}A_{0}B_{1}C_{0}\rangle{} - 2*\langle{}A_{0}B_{1}C_{1}\rangle{} + 2*\langle{}A_{1}B_{0}C_{0}\rangle{} + \langle{}A_{1}B_{0}C_{1}\rangle{} + \langle{}A_{1}B_{0}\rangle{}*\langle{}C_{1}\rangle{} + \langle{}B_{0}C_{0}\rangle{}*\langle{}C_{1}\rangle{} - \langle{}B_{0}\rangle{} - \langle{}C_{0}\rangle{}*\langle{}C_{1}\rangle{} - 3

### Equation F2

In [26]:
[pinf, [ynorm, ymarginal]] = set_problem_NS_C(BSMprob(0.9212), solve=True)

marginalsymb = np.zeros((oA,oA,oB,oC,mA,mA,mC), dtype=object)
for a2 in range(oA):
    for x2 in range(mA):
        marginalsymb[:,a2,:,:,:,x2,:] = symp * np.sum(symp[a2,:,:,x2,0])
            
sp.nsimplify(sp.expand((
                        8*(ymarginal @ marginalsymb.flatten() + sum(ynorm))
                        ).subs(zip(probs, corrs))), tolerance=1e-6)

The problem is infeasible. Getting certificate...
Primal or dual infeasibility certificate found


2*\langle{}A_{0}B_{1}C_{0}\rangle{} - 2*\langle{}A_{0}B_{1}C_{1}\rangle{} + \langle{}A_{1}B_{0}C_{0}\rangle{} + 2*\langle{}A_{1}B_{0}C_{1}\rangle{} + \langle{}A_{1}B_{0}\rangle{}*\langle{}A_{1}\rangle{} - \langle{}A_{1}\rangle{}**2 + \langle{}A_{1}\rangle{}*\langle{}B_{0}C_{1}\rangle{} + \langle{}A_{1}\rangle{}*\langle{}C_{0}\rangle{} - \langle{}A_{1}\rangle{}*\langle{}C_{1}\rangle{} - \langle{}B_{0}\rangle{} - 3